In [ ]:
import dcarte
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from matplotlib import colors
 
pd.options.mode.chained_assignment = None  
import seaborn as sns


In [ ]:
sorted_ids = pd.read_csv('sorted_ids_14_11_2023.csv')
sorted_ids = sorted_ids[['participant_id', 'Days Of Data',  'period_of collection','min_date','max_date']]
sorted_ids = sorted_ids.rename(columns={'min_date':'initial_date', 'max_date':'final_date'})
sorted_ids

In [ ]:
patient = dcarte.load('Patients','lookup',reload=False)

patient['participant_id'] = patient['id'].apply(lambda x:x[:5])
patient = patient[patient['participant_id'].isin(sorted_ids['participant_id'].unique())][['participant_id', 'birth_date', 'gender']]
patient = patient.rename(columns={'birth_date':'birth_year'})
patient

In [ ]:
demographics = pd.read_excel('redcap_demographics_diagnosis_nov16.xlsx')
demographics = demographics[['participant_id','Year of birth PwD', 'Gender PwD','Diagnosis', 'alzheimer_diagnosis']]
demographics.head(3)

In [ ]:
patient = patient.merge(demographics,on='participant_id',how='left')
patient 

In [ ]:
mmse = pd.read_excel("redcap_mmse_baseline_values_nov30.xlsx")
mmse = mmse[mmse['closest_to_initial_day']==True]
mmse = mmse.rename(columns={'days_after_min_date':'date_difference_with_first_date_mmse'})
mmse

In [ ]:
npi = pd.read_csv('npi_values_30_NOV_23_for_lme.csv')
npi = npi[npi['closest_to_initial_day']==True]
npi = npi.rename(columns={'time_in_study':'date_difference_with_first_date_npi','amended datetime':'date_test_npi'})
npi = npi[['participant_id','date_test_npi', 'date_difference_with_first_date_npi','min_date', 'npi_total', 'npi_sleep']]
npi

In [ ]:
patient = patient.merge(mmse[['participant_id', 'date_test_mmse', 'tot_score_mmse',
                              'date_difference_with_first_date_mmse','min_date']],on='participant_id',how='left')
patient = patient.merge(npi,on='participant_id',how='left')
patient

In [ ]:
final = sorted_ids.merge(patient,on='participant_id')
final

In [ ]:
difference_function = lambda row: pd.to_datetime(row['initial_date'],dayfirst=True).year - row['Year of birth PwD'] if pd.notnull(row['Year of birth PwD']) else pd.to_datetime(row['initial_date'],dayfirst=True).year - float(row['birth_year'])
final['age_ate_start'] = final.apply(difference_function, axis=1)
final['gender'] = final['gender'].replace('m', 'Man')
final['gender'] = final['gender'].replace('f', 'Woman')
final['Gender PwD'] = final['Gender PwD'].replace('Male', 'Man')
final['Gender PwD'] = final['Gender PwD'].replace('Female', 'Woman')
final

In [ ]:
final[['participant_id', 'Days Of Data', 'period_of collection',
       'initial_date', 'final_date', 'age_ate_start','Year of birth PwD', 'birth_year', 
      'gender','Gender PwD', 'Diagnosis', 'alzheimer_diagnosis',
       'date_test_mmse', 'tot_score_mmse',
       'date_difference_with_first_date_mmse','date_test_npi',
       'date_difference_with_first_date_npi', 'npi_sleep','npi_total', 'min_date_x','min_date_y']].to_excel('table_demographics_11_DEC_2023.xlsx')

In [ ]:
final['Days Of Data'].sum()

In [ ]:
final.columns